In [ ]:
import pycountry
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
df1 = pd.read_csv("energy_statistics_UE_v3.csv")
df1

In [ ]:
list_countries = df1['country_or_area'].unique().tolist()
list_countries

In [ ]:
d_country_code = {}  # To hold the country names and their ISO
for country in list_countries:
    try:
        country_data = pycountry.countries.search_fuzzy(country)
        # country_data is a list of objects of class pycountry.db.Country
        # The first item  ie at index 0 of list is best fit
        # object of class Country have an alpha_3 attribute
        country_code = country_data[0].alpha_3
        d_country_code.update({country: country_code})
    except:
        print('could not add ISO 3 code for ->', country)
        # If could not find country, make ISO code ' '
        d_country_code.update({country: ' '})
        
d_country_code       

In [ ]:
def mapping(el):
    iso_code = d_country_code.get(el)
    return iso_code

In [ ]:
df1["ISO_code"] = df1["country_or_area"].apply(mapping)
df1

In [ ]:
var_1 = df1["wide_category"] == "Green"
var_2 = df1["wide_category"] == "Nuclear"
var_3 = df1["wide_category"] == "Electricity_and_Heat"
df1 = df1[var_1 | var_2 | var_3]
df1 = df1[df1["unit"] == "Kilowatt-hours, million"]
df1 = df1[df1["transaction"].notna()]
df1 = df1[df1["transaction"].str.contains("production",case = False)]
df1 = df1[df1["transaction"] != "net production"] 
df1 = df1[df1["transaction"] != "total production, main activity"]
df1 = df1[df1["transaction"] != "total production, autoproducer"]
df1

In [ ]:
green = df1[df1["wide_category"] == "Green"]
grouped_green = green.groupby(by=["year","country_or_area", "ISO_code"], as_index = False)
gg = pd.DataFrame(grouped_green["quantity"].sum())
gg

In [ ]:
nuclear = df1[df1["wide_category"] == "Nuclear"]
grouped_nuclear = nuclear.groupby(by=["year","country_or_area", "ISO_code"], as_index = False)
gn = pd.DataFrame(grouped_nuclear["quantity"].sum())
gn

In [ ]:
electricity = df1[df1["wide_category"] == "Electricity_and_Heat"]
grouped_electricity = electricity.groupby(by=["year","country_or_area", "ISO_code"], as_index = False)
ge = pd.DataFrame(grouped_electricity["quantity"].sum())
ge

In [ ]:
category_list_map = list(df1["wide_category"].unique())
choice_widget_category_map = widgets.Dropdown(options=category_list_map)
values_map = {"Kategoria": choice_widget_category_map.value}
def widgets_handler_map(category_choice_map):
    values_map["Kategoria"] = category_choice_map
    
def widget_map():
    widgets.interact(widgets_handler_map, category_choice_map=choice_widget_category_map)


In [ ]:
def print_map():
    fig = px.choropleth(data_frame = df1[df1["wide_category"] == values_map["Kategoria"]].sort_values(by = "year"),
                        scope= "europe",
                        locations= "ISO_code",
                        color= "quantity",  # value in column 'quantity' determines color
                        hover_name= "country_or_area",
                        color_continuous_scale= 'RdYlGn',#  color scale red, yellow green
                        animation_frame= "year",
                        title = "{} energy produced in UE (MkWh)".format(values_map["Kategoria"]))
    fig.show()

In [ ]:
def print_map_green():
    fig = px.choropleth(data_frame = gg,
                        scope= "europe",
                        locations= "ISO_code",
                        color= "quantity",  # value in column 'quantity' determines color
                        hover_name= "country_or_area",
                        color_continuous_scale= 'RdYlGn',#  color scale red, yellow green
                        animation_frame= "year",
                        title = "Green energy produced in UE (MkWh)")
    fig.show()

In [ ]:
def print_map_nuclear():
    fig = px.choropleth(data_frame = gn,
                        scope= "europe",
                        locations= "ISO_code",
                        color= "quantity",  # value in column 'quantity' determines color
                        hover_name= "country_or_area",
                        color_continuous_scale= 'RdYlGn',#  color scale red, yellow green
                        animation_frame= "year",
                        title = "Nuclear energy produced in UE (MkWh)")
    fig.show()

In [ ]:
def print_map_electricity():
    fig = px.choropleth(data_frame = ge,
                        scope= "europe",
                        locations= "ISO_code",
                        color= "quantity",  # value in column 'quantity' determines color
                        hover_name= "country_or_area",
                        color_continuous_scale= 'RdYlGn',#  color scale red, yellow green
                        animation_frame= "year",
                        title = "Electricity produced in UE (MkWh)")
    fig.show()

In [ ]:
#wersja z widgetem
widget_map()
print_map()

In [ ]:
#wersja bez widgetu
print_map_green()
print_map_nuclear()
print_map_electricity()